# 📝 Celda 1 — Imports, rutas y semillas


Configuramos TensorFlow/Keras, rutas a secuencias del Cuaderno 4 y la semilla para reproducibilidad.

In [ ]:
from pathlib import Path
import itertools, json, time, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, Tuple

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reproducibilidad
SEED = 42
np.random.seed(SEED); tf.random.set_seed(SEED)

# Rutas de los datasets secuenciales (creados en Cuaderno 4)
SEQ_ROOT = Path("../data/processed/seq")

# Directorios de salida
FIG_DIR = Path("../reports/figures")
OUT_DIR = Path("../reports/models")
FIG_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("TF:", tf.__version__)


# 📝 Celda 2 — Carga de secuencias (train/val/test)

Utilidades para cargar X/y y metadatos de cada ticker y window_size.

In [ ]:
def load_seq(ticker: str, W: int):
    base = SEQ_ROOT / ticker / f"w{W}"
    Xtr = np.load(base / "X_train.npy"); ytr = np.load(base / "y_train.npy")
    Xva = np.load(base / "X_val.npy");   yva = np.load(base / "y_val.npy")
    Xte = np.load(base / "X_test.npy");  yte = np.load(base / "y_test.npy")
    with open(base / "meta.json", "r", encoding="utf-8") as f:
        meta = json.load(f)
    return (Xtr,ytr), (Xva,yva), (Xte,yte), meta

# Prueba rápida
for tkr in ["BBVA","SAN"]:
    for W in [10,20,30]:
        (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, W)
        print(f"{tkr} W={W} -> Xtr{Xtr.shape}, Xva{Xva.shape}, Xte{Xte.shape}, feats={meta['n_features']}")


# 📝 Celda 3 — Constructor de modelos (SimpleRNN / LSTM / GRU)

Función para construir el modelo con una sola capa recurrente y cabeza densa lineal (como en el TFG). Optimización con Adam y pérdida MSE.

In [ ]:
def build_model(model_type: str, units: int, n_features: int, window_size: int, lr: float) -> keras.Model:
    model = keras.Sequential(name=f"{model_type}_u{units}_w{window_size}_lr{lr}")
    if model_type == "SimpleRNN":
        model.add(layers.SimpleRNN(units, input_shape=(window_size, n_features)))
    elif model_type == "LSTM":
        model.add(layers.LSTM(units, input_shape=(window_size, n_features)))
    elif model_type == "GRU":
        model.add(layers.GRU(units, input_shape=(window_size, n_features)))
    else:
        raise ValueError("model_type debe ser 'SimpleRNN', 'LSTM' o 'GRU'")
    model.add(layers.Dense(1))
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss="mse", metrics=["mse"])
    return model


# 📝 Celda 4 — Entrenamiento y evaluación (con EarlyStopping)

Entrenamos con EarlyStopping (paciencia 2, monitor val_loss) y devolvemos historia, métricas de val/test y predicciones para test.

In [ ]:
def fit_and_eval(model_type: str, tkr: str, W: int, units: int, batch: int, lr: float, epochs: int):
    (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, W)
    model = build_model(model_type, units, meta["n_features"], W, lr)
    es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
    history = model.fit(
        Xtr, ytr,
        validation_data=(Xva,yva),
        epochs=epochs,
        batch_size=batch,
        verbose=0,
        callbacks=[es]
    )
    # Evaluación
    val_mse = model.evaluate(Xva, yva, verbose=0)[0]
    test_mse = model.evaluate(Xte, yte, verbose=0)[0]
    yhat_test = model.predict(Xte, verbose=0).ravel()
    return {
        "model": model,
        "history": history.history,
        "val_mse": float(val_mse),
        "test_mse": float(test_mse),
        "y_true_test": yte,
        "y_pred_test": yhat_test
    }


# 📝 Celda 5 — Curvas de pérdida (5/7/10 épocas) como en el TFG

Para cada modelo (SimpleRNN, LSTM, GRU) y cada ticker, dibujamos tres curvas de pérdida (train/val) con epochs = 5, 7, 10.
Usamos una configuración base units=64, batch=32, lr=1e-3 (mismo espíritu del TFG).

In [ ]:
BASE_CFG = dict(units=64, batch=32, lr=1e-3)
EPOCHS_LIST = [5,7,10]
MODELS = ["SimpleRNN","LSTM","GRU"]
TICKERS = ["BBVA","SAN"]
WINDOW_FOR_PLOTS = 20  # el TFG suele fijar una ventana para las curvas; puedes cambiar a 10 o 30 si prefieres

def plot_losses(histories, title):
    plt.figure(figsize=(8,5))
    for ep, h in histories.items():
        plt.plot(h["loss"], label=f"train (e={ep})")
        plt.plot(h["val_loss"], label=f"val (e={ep})", linestyle="--")
    plt.title(title); plt.xlabel("Época"); plt.ylabel("MSE (pérdida)")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

for tkr in TICKERS:
    for m in MODELS:
        hist_dict = {}
        for ep in EPOCHS_LIST:
            res = fit_and_eval(m, tkr, WINDOW_FOR_PLOTS, **BASE_CFG, epochs=ep)
            hist_dict[ep] = res["history"]
        plot_losses(hist_dict, f"{tkr} · {m} — Curvas de pérdida (W={WINDOW_FOR_PLOTS})")

        # (Opcional) guarda la última figura si quieres reportarla
        # plt.savefig(FIG_DIR / f"{tkr}_{m}_loss_curves_W{WINDOW_FOR_PLOTS}.png", dpi=130)


# 📝 Celda 6 — Grid Search (MSE en validación) y ranking

Aplicamos Grid Search (exhaustivo) con los rangos del TFG y guardamos un ranking por ticker y modelo.
La métrica de selección es val_mse. Al final, re-evaluamos la mejor combinación también en test.

In [ ]:
GRID = {
    "window": [10,20,30],
    "units": [32,64,128],
    "batch": [32,64],
    "lr": [1e-3, 5e-4],
}
MAX_EPOCHS = 10  # en el TFG se prueban 5/7/10; para el grid usamos 10 con EarlyStopping

def grid_search_for_model(tkr: str, model_type: str) -> pd.DataFrame:
    rows = []
    for W, U, B, LR in itertools.product(GRID["window"], GRID["units"], GRID["batch"], GRID["lr"]):
        res = fit_and_eval(model_type, tkr, W, units=U, batch=B, lr=LR, epochs=MAX_EPOCHS)
        rows.append({
            "ticker": tkr, "model": model_type,
            "window": W, "units": U, "batch": B, "lr": LR,
            "val_mse": res["val_mse"], "test_mse": res["test_mse"]
        })
        # Guardado incremental (por si se corta la ejecución)
        pd.DataFrame(rows).to_csv(OUT_DIR / f"grid_partial_{tkr}_{model_type}.csv", index=False)
    df_rank = pd.DataFrame(rows).sort_values(["val_mse","test_mse"]).reset_index(drop=True)
    return df_rank

all_ranks = []
for tkr in TICKERS:
    for m in MODELS:
        print(f"↳ Grid Search: {tkr} · {m}")
        rank_df = grid_search_for_model(tkr, m)
        rank_df.to_csv(OUT_DIR / f"grid_{tkr}_{m}.csv", index=False)
        all_ranks.append(rank_df.assign(order=range(1, len(rank_df)+1)))

grid_all = pd.concat(all_ranks, ignore_index=True)
grid_all.head()


# 📝 Celda 7 — Top-10 por modelo y ticker (tabla para el informe)

Mostramos y guardamos la tabla Top-10 (como en el TFG) para cada combinación ticker/model.

In [ ]:
tops = []
for tkr in TICKERS:
    for m in MODELS:
        dfm = grid_all[(grid_all["ticker"]==tkr) & (grid_all["model"]==m)].sort_values(["val_mse","test_mse"]).head(10)
        dfm.to_csv(OUT_DIR / f"top10_{tkr}_{m}.csv", index=False)
        print(f"\n=== {tkr} · {m} — Top-10 (por val_mse) ===")
        display(dfm)
        tops.append(dfm.assign(kind=f"{tkr}_{m}"))

top_all = pd.concat(tops, ignore_index=True)
top_all.to_csv(OUT_DIR / "top10_all.csv", index=False)


# 📝 Celda 8 — Selección del mejor por modelo (para las gráficas “real vs predicho”)

Elegimos la mejor configuración (mínimo val_mse) para cada ticker/model, reentrenamos (EarlyStopping) y graficamos real vs predicho en test.

In [ ]:
def best_config(df: pd.DataFrame, tkr: str, m: str):
    sub = df[(df["ticker"]==tkr) & (df["model"]==m)].sort_values(["val_mse","test_mse"]).head(1).iloc[0]
    return dict(W=int(sub["window"]), U=int(sub["units"]), B=int(sub["batch"]), LR=float(sub["lr"]))

def plot_real_pred(y_true, y_pred, title):
    plt.figure(figsize=(12,5))
    plt.plot(y_true, label="Real (Close t+1)", linewidth=1.2)
    plt.plot(y_pred, label="Predicho", linewidth=1.2)
    plt.title(title); plt.xlabel("Índice temporal (test)"); plt.ylabel("Close (escalado)")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

best_runs = []
for tkr in TICKERS:
    for m in MODELS:
        cfg = best_config(grid_all, tkr, m)
        res = fit_and_eval(m, tkr, cfg["W"], cfg["U"], cfg["B"], cfg["LR"], epochs=MAX_EPOCHS)
        plot_real_pred(res["y_true_test"], res["y_pred_test"], f"{tkr} · {m} — Mejor configuración (test)")
        best_runs.append({
            "ticker": tkr, "model": m, "window": cfg["W"], "units": cfg["U"],
            "batch": cfg["B"], "lr": cfg["LR"], "val_mse": res["val_mse"], "test_mse": res["test_mse"]
        })

best_table = pd.DataFrame(best_runs).sort_values(["ticker","test_mse"])
best_table.to_csv(OUT_DIR / "best_models_summary.csv", index=False)
best_table


# 📝 Celda 9 — Comparativa final de MSE (barras)

Gráfico de barras comparando el MSE de test de los mejores modelos (SimpleRNN vs LSTM vs GRU) para cada ticker.

In [ ]:
def bar_compare(best_table, ticker):
    sub = best_table[best_table["ticker"]==ticker].sort_values("test_mse")
    plt.figure(figsize=(6,4))
    plt.bar(sub["model"], sub["test_mse"])
    for i,(m,v) in enumerate(zip(sub["model"], sub["test_mse"])):
        plt.text(i, v, f"{v:.5f}", ha="center", va="bottom", fontsize=9)
    plt.title(f"{ticker} — MSE (test) mejores modelos")
    plt.ylabel("MSE (test)"); plt.grid(axis="y", alpha=0.2)
    plt.tight_layout(); plt.show()

for tkr in TICKERS:
    bar_compare(best_table, tkr)


# 📝 Celda 10 — Guardado opcional de modelos (H5)

Si quieres conservar los pesos de los mejores para incluirlos en el repo/entrega:

In [ ]:
SAVE_MODELS = True

if SAVE_MODELS:
    for row in best_table.itertuples(index=False):
        (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(row.ticker, int(row.window))
        model = build_model(row.model, int(row.units), meta["n_features"], int(row.window), float(row.lr))
        es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
        model.fit(Xtr, ytr, validation_data=(Xva,yva), epochs=MAX_EPOCHS, batch_size=int(row.batch), verbose=0, callbacks=[es])
        path = OUT_DIR / f"{row.ticker}_{row.model}_w{row.window}_u{row.units}_b{row.batch}_lr{row.lr}.h5"
        model.save(path)
        print("💾 Guardado:", path)


## 📝 Resumen



- Curvas de pérdida (5/7/10 épocas) para SimpleRNN, LSTM y GRU en BBVA y SAN (W=20).

- Grid Search con hiperparámetros del TFG:

window_size = 10, 20, 30

units = 32, 64, 128

batch_size = 32, 64

learning_rate = 0.001, 0.0005

- Métrica de selección: MSE (validación).

- Top-10 por ticker/model (CSV en reports/models).

- Mejores configuraciones reentrenadas y evaluadas en test con gráficas “real vs predicho”.

- Barras comparativas de MSE (SimpleRNN vs LSTM vs GRU) por ticker.

- Modelos guardados (.h5) para reproducibilidad.